# Thermal Image Enhancement Network (TEN) - IROS 2018
Thermal Image Enhancement using Convolution Neural Network

In [6]:
import torch
import torchvision
from torch import nn

import os

## Dataset

In [7]:
dataset_train_path = '/DATA1/chaitanya/ninad3/dataset/rgb91/'
assert(os.path.exists(dataset_train_path))

In [ ]:
class rgb91(Dataset):
    
    def __init__(self,path,transform=None):
        
        self.path = path
        self.transform = transform
    
    def __len__(self):
        
        return len(os.listdir(self.path))
    
    def __getitem__(self,idx):
        
        imgList = os.listdir(self.path)
        imgName = imgList[idx]
        
        imgPath = os.path.join(self.path,imgName)
        
        img = PIL.Image.open(imgPath)
               
        if self.transform is not None:
            img = self.transform(img)
       
        return (imgPath,img)

## Parameters

In [ ]:
batch_size = 128
learning_rate = 0.001
num_epochs = 100
weight_decay = 5e-4

## Model

In [9]:
class ten(nn.Module):
    
    model = nn.Sequential(
        
        nn.Conv2d(3,64,7,stride=1,padding=3),
        nn.ReLU(True),
        
        nn.Conv2d(64,32,5,stride=1,padding=2),
        nn.ReLU(True),
        
        nn.Conv2d(32,32,3,stride=1,padding=1),
        nn.ReLU(True),
        
        nn.Conv2d(32,1,3,stride=1,padding=1)
    )
    
    def forward(self,x):
        x = self.model(x)
        

In [ ]:
model = torch.nn.DataParallel(ten()).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, 
                             weight_decay=weight_decay)

## Methods

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
#     learning_rate = args.lr * (0.1 ** (epoch // 30))
    learning_rate = learning_rate/10
    logs.write(f'Learning Rate set to {learning_rate}\n')
    print(f'Learning Rate set to {learning_rate}')
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate